**Laboratorio de métodos computacionales**

**Universidad de los Andes**

**Profesor: Diego Alberto Castro Rodríguez**

## Integración con Metropolis-Hastings

Dada una distribuón $g(x)$ y una función $f(x)$ se tiene que

$$ \frac{\int g(x)f(x)dx}{\int g(x)dx} = \left< f(x) \right> \approx \frac{1}{N} \sum_{i=1}^{N} f(x_{i}) $$

Si se tiene una integral que se pueda escribir en la forma $\int g(x)f(x)dx$, se puede hacer uso del algoritmo de Metropolis-Hastings para encontrar el valor de dicha integral si se calcula o se sabe $\int g(x)dx$ (que suele llamarse factor de normalización)

$$I = \int g(x)f(x)dx \approx \frac{\int g(x)dx}{N}\sum_{i = 1}^{N}g(X)$$




**Ejemplo:** Use Metropolis-Hasting para estimar el valor de la integral

$$I = \int_{-\infty}^{\infty} \int_{-\infty}^{\infty}  x\frac{e^{-\frac{1}{2}(-8x-8y+x^2y^2+x^2+y^2)}}{C}dxdy$$

Donde la constante de normalización $C\approx 20216.336$

(1.905832263002595, 0.010690429534252893)

## Estimación de parámetros con Metropolis-Hastings

Suponga que tiene datos $D (t_i)$,  (ya sea un escalar o un vector) en puntos discretos $t_i$ y un modelo propuesto, que produce una serie $y (t | \theta)$, donde $\theta$ representa un conjunto de parámetros libres que cambia la función propuesta.

Lo primero que debe hacer es elegir una función de error. El más común es la suma de errores cuadrados dada por

$$\chi ^ 2 = \sum_i \frac {(D (t_i) -y (t_i | \theta)) ^ 2} {2 \sigma ^ 2}$$

donde $\sigma$ es una estimación del error de los datos. Usualmente, puede utilizar la incertidumbre experimental o simplemente adivinar.

Lo que se desea es que la probabilidad $P$ sea máxima cuando $\chi^2$ sea mínimo. Esto lo podemos plasmar por ejemplo así

$$
P(D | \theta) = e^{- \chi ^ 2}
$$

(Esto puede generar números que estan por fuera del rango de presición de python, si esto ocurre puede remplasar la función exponencial por otra que no cause este problema, o asegurarse de que sus valores iniciales para los parámetros no esten muy lejos de los valores buscados)

**Ejemplo:** Ajustar con un modelo lineal los datos generados con `t = np.linspace(0,10,num=11)` y con `D = 0.1*t + 7`. Ignorar los primeros 1000 samples.


